# Homework 2
In homework 2, we will work with log content of user in 30 days. The general idea here is we first try to analyze one day
then apply it to a month. Thanks to **pyspark**  supporting glob pattern directory, this can be done with ease. 


In [4]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, to_date, when, \
	input_file_name, regexp_extract, sum, lit

In [5]:
builder = (
	SparkSession.builder.appName("homework1")
	.config("spark.driver.memory", "16g")
	.config("spark.driver.cores", 4)
	.config("sql.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
	.config(
		"spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"
	)
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/10/25 20:56:06 WARN Utils: Your hostname, Andrea-Le-MBP-Pro.local resolves to a loopback address: 127.0.2.2; using 192.168.1.9 instead (on interface en0)
23/10/25 20:56:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/sonle/.sdkman/candidates/spark/3.4.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sonle/.ivy2/cache
The jars for the packages stored in: /Users/sonle/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6477a3f6-0ad4-4b44-9ab1-847172d4cf10;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 194ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Parquet is all the way
Log content here is stored on disk in **json format** which is not well-suited for reading purpose. 
Hence, we should load and write it down as **parquet format** since parquet is more toward to analytical workload. 

In [3]:
log_path = "/Users/sonle/Documents/GitHub/spark-playground/data/log_content"	
data_path = "/Users/sonle/Documents/GitHub/spark-playground/data/"
parquet_path = "parquet_log_content.parquet"

In [6]:
import os

os.path.exists(os.path.join(data_path, "log_content"))

True

In [ ]:
# Just run this function one time
def read_log_content(
		data_paths: str,
		parquet_path: str
):
	"""
    Read and rename log content data in json format and 
    write it to disk in parquet format with partitioning option
    :param data_paths: data directory
    :param parquet_path: parquet file name
    :return: None
    """
	return (
		spark.read.json(f"{data_paths}/log_content/*.json")
		.select(
			col("_index").alias("Index"),
			col("_type").alias("Type"),
			col("_id").alias("Id"),
			col("_score").alias("Score"),
			col("_source.*"),
			to_date(
				regexp_extract(input_file_name(), r"\d{8}", 0),
				"yyyyMMdd"
			).alias("Date")
		)
		.write.parquet(
			path=f"{data_paths}/{parquet_path}",
			mode="overwrite",
			partitionBy="Date",
			compression="zstd"
		)
	)


read_log_content(data_path, parquet_path)

In [ ]:
df = (
	spark.read.parquet(
		f"{data_path}/{parquet_path}"
	)
	.filter(col("Contract") != "0")
)


In [1]:
import datetime
def generate_date_range(start_date, end_date, format="%Y-%m-%d"):
  """Generates a range of dates in string format.

  Args:
    start_date: The start date of the range.
    end_date: The end date of the range.
    format: The format of the dates in the range.

  Returns:
    A list of dates in string format.
  """

  date_range = []
  current_date = start_date
  while current_date <= end_date:
    date_range.append(current_date.strftime(format))
    current_date += datetime.timedelta(days=1)
  return date_range

start_date = datetime.datetime(2022, 4, 1)
end_date = datetime.datetime(2022, 4, 3)
date_range = generate_date_range(start_date, end_date)
date_range

['2022-04-01', '2022-04-02', '2022-04-03']

In [11]:
list_file = [
	f"{data_path}{parquet_path}/Date={date}"
	for date in date_range
]
list_file

['/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet/Date=2022-04-01',
 '/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet/Date=2022-04-02',
 '/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet/Date=2022-04-03']

In [13]:
(
	spark.read.parquet(
		*list_file
	)
	.filter((col("Contract") != "0") & (col("Contract") != ""))
	.explain("formatted")
)

== Physical Plan ==
* Filter (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [8]: [Index#32, Type#33, Id#34, Score#35L, AppName#36, Contract#37, Mac#38, TotalDuration#39L]
Batched: true
Location: InMemoryFileIndex [file:/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet/Date=2022-04-01, ... 2 entries]
PushedFilters: [IsNotNull(Contract), Not(EqualTo(Contract,0)), Not(EqualTo(Contract,))]
ReadSchema: struct<Index:string,Type:string,Id:string,Score:bigint,AppName:string,Contract:string,Mac:string,TotalDuration:bigint>

(2) ColumnarToRow [codegen id : 1]
Input [8]: [Index#32, Type#33, Id#34, Score#35L, AppName#36, Contract#37, Mac#38, TotalDuration#39L]

(3) Filter [codegen id : 1]
Input [8]: [Index#32, Type#33, Id#34, Score#35L, AppName#36, Contract#37, Mac#38, TotalDuration#39L]
Condition : ((isnotnull(Contract#37) AND NOT (Contract#37 = 0)) AND NOT (Contract#37 = ))


In [ ]:
df.createOrReplaceTempView("log_content")


# ETL
The requirements at this part is that we need to extract the information of each contract in terms of total duration of five segments "TVDuration", "MovieDuration", "SportDuration",
"ChildDuration", "RelaxDuration".

The first option is to **aggregate sum function manually**, the second option is to use supported **pyspark pivot** method. 

Since we have 5 categories, we will need to rewrite some operations repeatedly which can take some time. For example:
```python
when(col("name") == "some_name", value).otherwise(value)
# or sum method in agg method
sum("columns").alias("columns_name")
```
Thankfully, pyspark functions support arguments unpacking `*exprs` so we can leverage this by creating lists of expressions before pasting them to pyspark functions.


In [ ]:
 app_names = [
        "CHANNEL",
		"KPLUS",
        "VOD",
        "FIMS",
        "BHD",
        "SPORT",
        "CHILD",
        "RELAX",
    ]

column_names = [
        "TV",
        "TV",
        "Movie",
        "Movie",
        "Movie",
        "Sport",
        "Child",
        "Relax",
    ]
# whens = [
# 	when(col("AppName").isin(app_name), column_name).otherwise("")
# 	for app_name, column_name in zip(app_names, column_names)
# ]
# whens
import pyspark.sql.functions as F

whens_1 = F
for app_name, column_name in zip(app_names, column_names):
	whens_1 = whens_1.when(col("AppName") == app_name, column_name)
whens_1 = whens_1.otherwise("").alias("Type")
whens_1

	

In [ ]:
def summarize_by_manual_pivot(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str]
) -> DataFrame:
	"""
	Function to manually pivot data. 
	:param df: Dataframe, log content data
	:param app_names: application names in original data
	:param column_names: new column names after aggregation
	:return: new summarized data with required format
	"""
	
	whens = [when(col("AppName") == app_name, col("TotalDuration")).otherwise(lit(0)).alias(f"{column_name}")
			 for app_name, column_name in zip(app_names, column_names)
			 ]
	
	exprs = [sum(x).alias(f"{x}") for x in column_names]

	return (
		df
		.select(
			col("Contract"),
			*whens)
		.groupby("Contract")
		.agg(*exprs)
		.orderBy("TVDuration", ascending=False)
	)


In [ ]:
%%time
summarize_by_manual_pivot(df, app_names=app_map, column_names=columns_name)

In [ ]:
spark.sql(
	sqlQuery="""
	select
		distinct  AppName
		
	from log_content 
	"""
).show()

In [ ]:

(
	df
	.select(
		col("Contract"),
		col("TotalDuration"),
		whens_1
	)
	.filter(
        (col("Contract") != "0") & (col("Contract") != "") & (col("Type") != "Unknown")
	)
	.groupBy("Contract")
	.pivot("Type")
	.sum("TotalDuration")
	.show()
)



In [ ]:
whens_1

In [ ]:
df.show() # HNH433136,null,4886,null,null,2034901


In [ ]:
(
	df
		.groupby("Contract")
		.pivot("Type")
		.sum("TotalDuration")
		.show(truncate=False)
)

In [ ]:
def summarize_by_supported_pivot(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str]
) -> DataFrame:
	"""
	Function to pivot data using supported pyspark one
	:param df: 
	:param app_names: 
	:param column_names: 
	:return: 
	"""
	exprs = [col(x).alias(f"{y}") for x, y in zip(app_names, column_names)]

	return (
		df
		.groupby("Contract")
		.pivot("AppName", app_names)
		.sum("TotalDuration")
		.select(
			col("Contract"),
			*exprs
		)
		.orderBy("TVDuration", ascending=False)
	)

In [ ]:
%%time
summarize_by_supported_pivot(df, app_names=app_names, column_names=column_names)
